# Web Scraping for Indeed.com & Predicting Salaries

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com. In the second, we'll focus on using listings with salary information to build a model and predict additional salaries.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

The URL here has many query parameters

- `q` for the job search
- This is followed by "+20,000" to return results with salaries (or expected salaries >$20,000)
- `l` for a location 
- `start` for what result number to start on

In [ ]:

#URL = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

In [ ]:
YOUR_CITY = ''

In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib2
import json

import pandas as pd

import re
import requests

from sklearn.model_selection import cross_val_score
import patsy
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.grid_search import GridSearchCV



Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

In [377]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
max_results_per_city = 1000 # Set this to a high-value (5000) to generate more results. 
# Crawling more results, will also take much longer. First test your code on a small number of results and then expand.

results = []

for city in ['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami']:
    for start in range(0, max_results_per_city, 10):
        url_ = 'http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l='+city+'&start='+str(start)
        soupy = (BeautifulSoup(urllib2.urlopen(url_).read()))
        results.append(soupy.find_all('div', class_=' row result'))

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- **Make sure these functions are robust and can handle cases where the data/field may not be available.**
    - Remember to check if a field is empty or `None` for attempting to call methods on it
    - Remember to use `try/except` if you anticipate errors
- **Test** the functions on the results above and simple examples

In [378]:
## YOUR CODE HERE
def get_location(result):
    di = {}
    for id1, div in enumerate(result):
        for id2, span_loc in enumerate(div.find_all('span', class_='location')):
            di[id2] = span_loc.text
    return di

def get_company(result):
    di = {}
    for id1, div in enumerate(result):
        for id2, span_comp in enumerate(div.find_all('span', class_='company')):
            di[id2] = span_comp.text
    return di
        
def get_jobtitle(result):
    di = {}
    for id1, div in enumerate(result):
        for id2, header in enumerate(div.find_all('h2')):
            di[id2] = header.text
    return di

def get_salary(result):
    di = {}
    for id1, div in enumerate(result):
        for id2, td_snip in enumerate(div.find_all('td', class_='snip')):
            for id3, nobr in enumerate(td_snip.find_all('nobr')):
                di[id2] = nobr.text
    return di

In [379]:
li = []
for i in results:
    li.append(get_location(i))
df = pd.DataFrame(li)

In [380]:
li = []
for i in results:
    li.append(get_jobtitle(i))
df['job'] = pd.DataFrame(li)

In [381]:
li = []
for i in results:
    li.append(get_salary(i))
df['salary'] = pd.DataFrame(li)

In [382]:
li = []
for i in results:
    li.append(get_company(i))
df['company'] = pd.DataFrame(li)


In [383]:
df.columns = ['city', 'job', 'salary', 'company']
df

,city,job,salary,company
0,"New York, NY",\nData Scientist\n,NaN,\n\n pulseData\n
1,"New York, NY",\nData Scientist\n,NaN,\n\n\n PlaceIQ\n
2,"New York, NY","\nJunior Data Scientist, Data & Statistics - D...",$90 an hour,\n\n\n Federal Reserve Bank of New York\n
3,"New York, NY 10036 (Clinton area)",\nMachine Learning and Data Science Experts\n,NaN,\n\n Gambit Technologies\n
4,"New York, NY",\nVP - Predictive Modeling / Data Scientist\n,"$80,000 - $200,000 a year",\n\n 3coast\n
5,"New York, NY 10004 (Financial District area)",\nChief Data Scientist\n,"$90,000 - $110,000 a year",\n\n Medivo Inc.\n
6,"New York, NY","\nSoftware Architect, Sports Data\n",NaN,\n\n\n Major League Baseball Advanced M...
7,"New York, NY 10016 (Gramercy area)",\nMachine Learning Data Scientist\n,NaN,\n\n\n Gilt Groupe\n
8,"New York, NY",\nField Data Scientist\n,NaN,\n\n\n Praescient Analytics\n
9,"New York, NY",\nData Scientist for a growing cyber security ...,"$110,000 - $140,000 a year",\n\n\n Workbridge Associates\n


Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

#### Complete the following code to collect results from multiple cities and starting points. 
- Enter your city below to add it to the search
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [ ]:
## YOUR CODE HERE
#did above

Lastly, we need to clean up salary data. 

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, these will not be useful to us for now
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

#### Find the entries with annual salary entries, by filtering the entries without salaries or salaries that are not yearly (filter those that refer to hour or week). Also, remove duplicate entries

In [385]:
#removing duplicates and drop nan's
df.dropna(how='any', axis=0, inplace=True)
df = df.drop_duplicates()
df.head()

/Users/dnay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,city,job,salary,company
2,"New York, NY","\nJunior Data Scientist, Data & Statistics - D...",$90 an hour,\n\n\n Federal Reserve Bank of New York\n
4,"New York, NY",\nVP - Predictive Modeling / Data Scientist\n,"$80,000 - $200,000 a year",\n\n 3coast\n
5,"New York, NY 10004 (Financial District area)",\nChief Data Scientist\n,"$90,000 - $110,000 a year",\n\n Medivo Inc.\n
9,"New York, NY",\nData Scientist for a growing cyber security ...,"$110,000 - $140,000 a year",\n\n\n Workbridge Associates\n
11,"New York, NY 10154 (Midtown area)",\nCognitive AI Data Scientist\n,"$33,650 a year",\n\n\n KPMG\n


#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [386]:
## YOUR CODE HERE
#salary column clean up
li = []
for i in df['salary']:
    li.append(str(i).replace('$', '').replace('a week', '').replace('an hour', '').replace(' a month', '')
                .replace(',', '').replace('a year', '').replace('-', ''))

li2 = [i.split() for i in li]
num_list = []
for id1, num in enumerate(li2):
    for nums in num:
        num_list += [(id1, float(nums))]
#print num_list
        
di = {}
for id1, num in num_list:
    if id1 not in di:
        di[id1] = num
    else:
        di[id1] += num
        di[id1] /= 2


for value in di:
    if di[value] < 100:
        di[value] *= (52*40)
    elif di[value] < 3000:
        di[value] *= 52
    elif di[value] < 10000:
        di[value] *= 12
salaries = []
for i in range(len(di)):
    salaries.append(di[i])

In [387]:
df['salary'] = salaries

In [388]:
#had to deal with unicode in order to send to csv, with both company and job title
li = []
for i in df['job']:
    li.append(str(i.encode('ascii', 'ignore')).split())
di1 = {}
for id1, string in enumerate(li):
    for letters in string:
        #skip newlines
        if '\\n' in letters:
            continue
        elif id1 not in di1:
            di1[id1] = str(letters.encode('ascii', 'ignore'))
        else:
            di1[id1] += ' ' + str(letters)
li2 = []
for i in range(len(di1)):
    li2.append(di1[i])
df['job'] = li2


listyx = []
for i in df['company']:
    listyx.append(str(i).split())
dictyx = {}
for id1, i in enumerate(listyx):
    for j in i:
        if '\\n' in j:
            continue
        elif id1 not in dictyx:
            dictyx[id1] = str(j)
        else:
            dictyx[id1] += ' ' + str(j)
listyxx = []
for i in range(len(dictyx)):
    listyxx.append(dictyx[i])
df['company'] = listyxx
df.head()

,city,job,salary,company
2,"New York, NY","Junior Data Scientist, Data & Statistics - Dat...",187200.0,Federal Reserve Bank of New York
4,"New York, NY",VP - Predictive Modeling / Data Scientist,140000.0,3coast
5,"New York, NY 10004 (Financial District area)",Chief Data Scientist,100000.0,Medivo Inc.
9,"New York, NY",Data Scientist for a growing cyber security co...,125000.0,Workbridge Associates
11,"New York, NY 10154 (Midtown area)",Cognitive AI Data Scientist,33650.0,KPMG


In [473]:
list1 = []
for i in df['city']:
    list1.append(str(i).split(',')[0])
df['city'] = list1
df.head()

,city,job,salary,company,high/low salary,is_high,developer,junior,senior,machine,python,manager,associate,scientist,data,research,analyst,predicted
2,New York,"Junior Data Scientist, Data & Statistics - Dat...",187200.0,Federal Reserve Bank of New York,very_high,1,0,1,0,0,0,0,0,1,1,0,0,very_high
4,New York,VP - Predictive Modeling / Data Scientist,140000.0,3coast,very_high,1,0,0,0,0,0,0,0,1,1,0,0,very_high
5,New York,Chief Data Scientist,100000.0,Medivo Inc.,high,1,0,0,0,0,0,0,0,1,1,0,0,very_high
9,New York,Data Scientist for a growing cyber security co...,125000.0,Workbridge Associates,high,1,0,0,0,0,0,0,0,1,1,0,0,very_high
11,New York,Cognitive AI Data Scientist,33650.0,KPMG,very_low,0,0,0,0,0,0,0,0,1,1,0,0,very_high


### Save your results as a CSV

In [389]:
df.to_csv('Indeed Job Data')

## Predicting salaries using Logistic Regression

#### Load in the the data of scraped salaries

In [ ]:
## YOUR CODE HERE

#### We want to predict a binary variable - whether the salary was low or high. Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

We could also perform Linear Regression (or any regression) to predict the salary value here. Instead, we are going to convert this into a _binary_ classification problem, by predicting two classes, HIGH vs LOW salary.

While performing regression may be better, performing classification may help remove some of the noise of the extreme salaries. We don't have to choice the `median` as the splitting point - we could also split on the 75th percentile or any other reasonable breaking point.

In fact, the ideal scenario may be to predict many levels of salaries, 

In [460]:
## YOUR CODE HERE
import numpy as np

#reasonable breaking points,
quartile1 = df['salary'].quantile(.25)
salary_median = np.median(df['salary'])
quartile3 = df['salary'].quantile(.75)
high_low_sal = []
for i in df['salary']:
    if i < quartile1:
        high_low_sal.append('very_low')
    elif i < salary_median:
        high_low_sal.append('low')
    elif i < quartile3:
        high_low_sal.append('high')
    else:
        high_low_sal.append('very_high')
df['high/low salary'] = high_low_sal
df.head()

listyx = []
for i in df['salary']:
    if i > salary_median:
        listyx.append(1)
    else:
        listyx.append(0)
df['is_high'] = listyx

#### Thought experiment: What is the baseline accuracy for this model?

In [461]:
## YOUR CODE HERE
# Baseline is the most common occurence
print df['high/low salary'].value_counts(), '\n'
print "Baseline: ",61. / (61+57+57+54)

very_high    61
very_low     57
high         57
low          54
Name: high/low salary, dtype: int64 

Baseline:  0.266375545852


#### Create a Logistic Regression model to predict High/Low salary using statsmodel. Start by ONLY using the location as a feature. Display the coefficients and write a short summary of what they mean.

In [462]:
## YOUR CODE HERE
import statsmodels.formula.api as sm
model = sm.logit(
    "is_high ~ city",
    data = df
).fit()

model.summary()

         Current function value: 0.354709
         Iterations: 35


/Users/dnay/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                is_high   No. Observations:                  229
Model:                          Logit   Df Residuals:                      122
Method:                           MLE   Df Model:                          106
Date:                Mon, 31 Oct 2016   Pseudo R-squ.:                  0.4882
Time:                        06:46:45   Log-Likelihood:                -81.228
converged:                      False   LL-Null:                       -158.71
                                        LLR p-value:                  0.001369
=============================================================================================================================
                                                                coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------------------------------------------
Intercept                                                    16.4969   3821.790      0.004      0.997     -7474.073  7507.067
city[T.Alpharetta, GA]                                      -36.5913   2.34e+04     -0.002      0.999     -4.59e+04  4.58e+04
city[T.Atlanta, GA]                                         -16.0915   3821.790     -0.004      0.997     -7506.662  7474.479
city[T.Atlanta, GA 30309 (Midtown area)]                      6.5496   1.01e+05   6.48e-05      1.000     -1.98e+05  1.98e+05
city[T.Atlanta, GA 30318 (Rockdale area)]                     6.5496   1.01e+05   6.48e-05      1.000     -1.98e+05  1.98e+05
city[T.Atlanta, GA 30328]                                   -36.5913   2.34e+04     -0.002      0.999     -4.59e+04  4.58e+04
city[T.Atlanta, GA 30338]                                   -36.5913   2.34e+04     -0.002      0.999     -4.59e+04  4.58e+04
city[T.Aurora, CO]                                          -16.4969   3821.790     -0.004      0.997     -7507.068  7474.074
city[T.Austin, TX]                                          -17.3442   3821.790     -0.005      0.996     -7507.915  7473.226
city[T.Beaverton, OR]                                       -38.4125   4.08e+04     -0.001      0.999     -7.99e+04  7.99e+04
city[T.Bellevue, WA]                                        -17.1901   3821.790     -0.004      0.996     -7507.761  7473.381
city[T.Bellevue, WA 98004 (Downtown area)]                  -36.5913   2.34e+04     -0.002      0.999     -4.59e+04  4.58e+04
city[T.Berkeley, CA]                                        -36.5913   2.34e+04     -0.002      0.999     -4.59e+04  4.58e+04
city[T.Berkeley, CA 94720]                                    7.6521   1.01e+05   7.55e-05      1.000     -1.99e+05  1.99e+05
city[T.Beverly Hills, CA]                                   -36.5913   2.34e+04     -0.002      0.999     -4.59e+04  4.58e+04
city[T.Boulder, CO]                                         -17.1901   3821.790     -0.004      0.996     -7507.761  7473.381
city[T.Brea, CA 92821]                                      -38.4126   4.08e+04     -0.001      0.999     -7.99e+04  7.99e+04
city[T.Brooklyn, NY 11216 (Bedford-Stuyvesant area)]          6.5496   1.01e+05   6.48e-05      1.000     -1.98e+05  1.98e+05
city[T.Broomfield, CO]                                      -38.4126   4.08e+04     -0.001      0.999     -7.99e+04  7.99e+04
city[T.Burbank, CA]                                           6.5496   1.01e+05   6.48e-05      1.000     -1.98e+05  1.98e+05
city[T.Centennial, CO 80111]                                -36.5913   2.34e+04     -0.002      0.999     -4.59e+04  4.58e+04
city[T.Chicago, IL]                                         -15.6860   3821.790     -0.004      0.997     -7506.256  7474.884
city[T.Chicago, IL 60290 (Loop area)]                         6.5496   1.01e+05   6.48e-05 

#### Create a few new variables in your dataframe to represent interesting features of a job title.
- For example, create a feature that represents whether 'Senior' is in the title 
- or whether 'Manager' is in the title. 
- Then build a new Logistic Regression model with these features. Do they add any value? 


In [465]:
def job_words(word):    
    temp = []
    for i in df['job']:
        if word in i.lower():
            temp.append(1)
        else:
            temp.append(0)
    df[word] = temp
def company_words(word):
    temp = []
    for i in df['job']:
        if word in i.lower():
            temp.append(1)
        else:
            temp.append(0)
    df[word] = temp
jobwords = ['manager', 'senior', 'research', 'machine', 'junior',
         'python', 'developer', 'analyst', 'scientist','data','associate']
# companywords = ['llc','bank','education','university','ibm','amazon','court']
for i in jobwords:
    job_words(i)
# for i in companywords:
#     company_words(i)

In [471]:
df

,city,job,salary,company,high/low salary,is_high,developer,junior,senior,machine,python,manager,associate,scientist,data,research,analyst,predicted
2,"New York, NY","Junior Data Scientist, Data & Statistics - Dat...",187200.0,Federal Reserve Bank of New York,very_high,1,0,1,0,0,0,0,0,1,1,0,0,very_high
4,"New York, NY",VP - Predictive Modeling / Data Scientist,140000.0,3coast,very_high,1,0,0,0,0,0,0,0,1,1,0,0,very_high
5,"New York, NY 10004 (Financial District area)",Chief Data Scientist,100000.0,Medivo Inc.,high,1,0,0,0,0,0,0,0,1,1,0,0,very_high
9,"New York, NY",Data Scientist for a growing cyber security co...,125000.0,Workbridge Associates,high,1,0,0,0,0,0,0,0,1,1,0,0,very_high
11,"New York, NY 10154 (Midtown area)",Cognitive AI Data Scientist,33650.0,KPMG,very_low,0,0,0,0,0,0,0,0,1,1,0,0,very_high
14,"New York, NY",Research Analyst,175000.0,Cushman & Wakefield,very_high,1,0,0,0,0,0,0,0,0,0,1,1,very_high
16,"New York, NY",Post-Doctoral Research Associate - Chen Group,150000.0,Central Office,very_high,1,0,0,0,0,0,0,1,0,0,1,0,very_high
17,"New York, NY",Lead Data Scientist - Top Data Consultancy,140000.0,Averity,very_high,1,0,0,0,0,0,0,0,1,1,0,0,very_high
18,"Hoboken, NJ",Data Engineer,150000.0,Jet,very_high,1,0,0,0,0,0,0,0,0,1,0,0,very_high
19,"Rahway, NJ",Data Scientist Job,220000.0,Merck,very_high,1,0,0,0,0,0,0,0,1,1,0,0,very_high


In [475]:
model = sm.logit(
    "is_high ~ city + developer + senior",
    data = df
).fit()

model.summary()

         Current function value: inf
         Iterations: 35


LinAlgError: Singular matrix

#### Rebuild this model with scikit-learn.
- You can either create the dummy features manually or use the `dmatrix` function from `patsy`
- Remember to scale the feature variables as well!


In [472]:
## YOUR CODE HERE
## YOUR CODE HERE

X = patsy.dmatrix('~ C(city) + C(manager) + C(senior) + + C(junior) +C(research) + C(associate) \
                  + C(machine) + C(python) + C(developer) + C(analyst) + C(scientist)', df)
y = df['high/low salary']

#### Use cross-validation in scikit-learn to evaluate the model above. 
- Evaluate the accuracy, AUC, precision and recall of the model. 
- Discuss the differences and explain when you want a high-recall or a high-precision model in this scenario.

In [420]:
## YOUR CODE HERE
logreg = LogisticRegression()
C_vals = [0.0001, 0.001, 0.01, 0.1, 0.5, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]
penalties = ['l1','l2']

gs = GridSearchCV(logreg, {'penalty':penalties, 'C':C_vals}, verbose=True, cv=5, scoring='f1_macro')
gs.fit(X, y)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


/Users/dnay/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:    1.1s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.0001, 0.001, 0.01, 0.1, 0.5, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring='f1_macro',
       verbose=True)

In [421]:
gs.best_params_

{'C': 0.0001, 'penalty': 'l2'}

In [422]:
logreg = LogisticRegression(C=1.0, penalty='l2', solver='liblinear')
logreg.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [423]:

cross_val_score(logreg, X, y)

array([ 0.47435897,  0.35526316,  0.41333333])

In [424]:
y_pred = logreg.predict(X)
print(classification_report(y, y_pred))
print pd.DataFrame(confusion_matrix(y, y_pred), index=['is_very_low','is_low', 'is_high', 'is_very_high'],
                   columns=['pred_very_low','pred_low', 'pred_high', 'pred_very_high'])

             precision    recall  f1-score   support

       high       0.74      0.47      0.57        73
  very_high       0.64      0.94      0.76       121
   very_low       1.00      0.11      0.21        35

avg / total       0.72      0.66      0.62       229



ValueError: Shape of passed values is (3, 3), indices imply (4, 4)

In [430]:
print 'coefficients:\n', logreg.coef_

coefficients:
[[ -4.02960344e-01   5.83296771e-01  -2.29923341e-01  -2.38938862e-01
   -2.53689406e-01  -2.38938862e-01   5.83296771e-01   2.77648629e-01
    6.59706444e-01   9.54265290e-01   6.67255726e-02   5.55170780e-01
   -2.38938862e-01  -6.10410609e-01   5.55170780e-01   6.76222218e-01
    9.60617233e-01  -2.38938862e-01   2.82169108e-01  -2.05733954e-01
    4.77923405e-01  -6.00095950e-01  -3.27957768e-01  -2.38938862e-01
   -2.28735969e-01  -4.48837705e-01   1.91581406e-01  -4.37636392e-01
   -3.27957768e-01  -3.27957768e-01  -2.56814623e-01   8.82487215e-01
    2.66927415e-01  -2.53689406e-01   1.29346420e-01  -2.24388615e-01
    5.42233800e-01  -2.38938862e-01  -2.56814623e-01  -4.55007397e-01
    4.96771528e-01  -2.53689406e-01   5.60930767e-01  -2.56814623e-01
   -2.72156301e-01   3.90743420e-02  -2.56814623e-01  -2.38938862e-01
   -2.61507832e-01  -2.38938862e-01  -1.73496343e-01   5.60930767e-01
    1.92814749e-01  -3.16286817e-01  -1.92063727e-01  -2.38938862e-01
   -4.

#### Compare L1 and L2 regularization for this logistic regression model. What effect does this have on the coefficients learned?

In [426]:
## YOUR CODE HERE
logreg = LogisticRegression(C=1.0, penalty='l1', solver='liblinear')
logreg.fit(X, y)
y_pred = logreg.predict(X)
print 'coefficients:', logreg.coef_
print(classification_report(y, y_pred))
print pd.DataFrame(confusion_matrix(y, y_pred), index=['is_very_low','is_low', 'is_high', 'is_very_high'],
                   columns=['pred_very_low','pred_low', 'pred_high', 'pred_very_high'])

coefficients: [[-0.33023916  0.          0.          0.          0.          0.          0.
   0.          0.43694894  0.84118628  0.          0.          0.          0.
   0.          0.14293571  0.8934647   0.          0.          0.          0.
  -0.37956519  0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.67353312  0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.00412712  0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   1.81987359  0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.         

ValueError: Shape of passed values is (3, 3), indices imply (4, 4)

In [427]:
## YOUR CODE HERE
logreg = LogisticRegression(C=1.0, penalty='l2', solver='liblinear')
logreg.fit(X, y)
y_pred = logreg.predict(X)
print 'coeficients:', logreg.coef_
print(classification_report(y, y_pred))
print pd.DataFrame(confusion_matrix(y, y_pred), index=['is_very_low','is_low', 'is_high', 'is_very_high'],
                   columns=['pred_very_low','pred_low', 'pred_high', 'pred_very_high'])

coeficients: [[ -4.02960344e-01   5.83296771e-01  -2.29923341e-01  -2.38938862e-01
   -2.53689406e-01  -2.38938862e-01   5.83296771e-01   2.77648629e-01
    6.59706444e-01   9.54265290e-01   6.67255726e-02   5.55170780e-01
   -2.38938862e-01  -6.10410609e-01   5.55170780e-01   6.76222218e-01
    9.60617233e-01  -2.38938862e-01   2.82169108e-01  -2.05733954e-01
    4.77923405e-01  -6.00095950e-01  -3.27957768e-01  -2.38938862e-01
   -2.28735969e-01  -4.48837705e-01   1.91581406e-01  -4.37636392e-01
   -3.27957768e-01  -3.27957768e-01  -2.56814623e-01   8.82487215e-01
    2.66927415e-01  -2.53689406e-01   1.29346420e-01  -2.24388615e-01
    5.42233800e-01  -2.38938862e-01  -2.56814623e-01  -4.55007397e-01
    4.96771528e-01  -2.53689406e-01   5.60930767e-01  -2.56814623e-01
   -2.72156301e-01   3.90743420e-02  -2.56814623e-01  -2.38938862e-01
   -2.61507832e-01  -2.38938862e-01  -1.73496343e-01   5.60930767e-01
    1.92814749e-01  -3.16286817e-01  -1.92063727e-01  -2.38938862e-01
   -4.8

ValueError: Shape of passed values is (3, 3), indices imply (4, 4)

#### Continue to incorporate other text features from the title or summary that you believe will predict the salary and examine their coefficients

#### Take ~100 scraped entries with salaries. Convert them to use with your model and predict the salary - which entries have the highest predicted salaries?

In [429]:
df['predicted'] = y_pred
highest_paying = df.ix[df['predicted'] == 'very_high'][['job','salary']]
print 'Top 10 highest paying jobs', '\n', highest_paying.sort_values('salary', ascending=False)[:10]
df.to_csv('Predicted Salaries')

Top 10 highest paying jobs 
                                                    job    salary
633        2017 Pharm Sci Intern-Analytical Science Job  250000.0
1303                      Senior Analyst/ Data engineer  250000.0
31                                       Data Scientist  250000.0
621   Associate Scientist/Study Director (Viral Clea...  220000.0
19                                   Data Scientist Job  220000.0
44          Clinical Research Assistant II - Psychiatry  215000.0
63                                   Front End Engineer  212500.0
255                  Senior Generalist Python Developer  200000.0
88                                Scientific Programmer  200000.0
77                   Geologist/ Environmental Scientist  200000.0


### BONUS 

#### Bonus: Use Count Vectorizer from scikit-learn to create features from the text summaries. 
- Examine using count or binary features in the model
- Re-evaluate the logistic regression model using these. Does this improve the model performance? 
- What text features are the most valuable? 

In [ ]:
## YOUR CODE HERE

In [ ]:
## YOUR CODE HERE

#### Re-test L1 and L2 regularization. You can use LogisticRegressionCV to find the optimal reguarlization parameters. 
- Re-test what text features are most valuable.  
- How do L1 and L2 change the coefficients?

In [ ]:
## YOUR CODE HERE